In [27]:
# we'll start off by importing necessary packages/libraries
# then, we'll scrape the statcast webpage for the data we need in this analysis
# this seems to be the best approach since the size of the CSV files makes manual downloading too slow and difficult
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [28]:
url = 'https://baseballsavant.mlb.com/statcast_search?hfPT=&hfAB=&hfGT=R%7C&hfPR=&hfZ=&hfStadium=&hfBBL=&hfNewZones=&hfPull=&hfC=&hfSea=2023%7C2022%7C2021%7C2020%7C2019%7C2018%7C&hfSit=&player_type=batter&hfOuts=&hfOpponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt=&game_date_lt=&hfMo=&hfTeam=&home_road=&hfRO=&position=&hfInfield=&hfOutfield=&hfInn=&hfBBT=&hfFlag=&metric_1=&group_by=team-year&min_pitches=0&min_results=0&min_pas=0&sort_col=pitches&player_event_sort=api_p_release_speed&sort_order=desc&chk_stats_pa=on&chk_stats_abs=on&chk_stats_bip=on&chk_stats_hits=on&chk_stats_singles=on&chk_stats_dbls=on&chk_stats_triples=on&chk_stats_hrs=on&chk_stats_so=on&chk_stats_k_percent=on&chk_stats_bb=on&chk_stats_bb_percent=on&chk_stats_ba=on&chk_stats_obp=on&chk_stats_slg=on&chk_stats_launch_speed=on&chk_stats_launch_angle=on&chk_stats_bbdist=on&chk_stats_hardhit_percent=on&chk_stats_barrels_per_pa_percent=on#results'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [29]:
table = soup.find_all('table')
table_titles = soup.find_all('th')

In [30]:
table_title_text = [title.text.strip() for title in table_titles]
table_title_text.pop(1)
table_title_text.pop()
#print(table_title_text)

''

In [31]:
league_batter_stats = pd.DataFrame(columns = table_title_text)
trash = []
print(league_batter_stats)

Empty DataFrame
Columns: [Rk., Player, Year, Pitches, Total, Pitch %, PA, AB, BIP, Hits, 1B, 2B, 3B, HR, SO, K%, BB, BB%, BA, OBP, SLG, EV (MPH), LA (°), Dist (ft), Hard Hit%, Barrel/PA%]
Index: []

[0 rows x 26 columns]


In [32]:
column_data = soup.find_all('tr')

In [33]:
for row in column_data[1:]:
    row_data = row.find_all('td')
    individual_row_data = [data.text.strip() for data in row_data]
    if len(individual_row_data) <= 1:
        trash.append(individual_row_data)
    else:
        individual_row_data.pop()
        individual_row_data.pop(1)
        length = len(league_batter_stats)
        league_batter_stats.loc[length] = individual_row_data

In [34]:
print(league_batter_stats)
print(trash)

     Rk. Player  Year Pitches   Total Pitch %    PA    AB   BIP  Hits  ...  \
0      1    LAD  2018  25,643  25,643   100.0  6311  5572  4214  1394  ...   
1      2    SEA  2019  25,523  25,523   100.0  6192  5500  3970  1305  ...   
2      3    BOS  2019  25,427  25,427   100.0  6439  5770  4452  1554  ...   
3      4    MIL  2019  25,312  25,312   100.0  6267  5542  4037  1366  ...   
4      5    NYY  2019  25,306  25,306   100.0  6227  5583  4189  1493  ...   
..   ...    ...   ...     ...     ...     ...   ...   ...   ...   ...  ...   
175  176    MIA  2020    8547    8547   100.0  2161  1935  1413   472  ...   
176  177    TEX  2020    8428    8428   100.0  2144  1936  1408   420  ...   
177  178    PIT  2020    8352    8352   100.0  2128  1932  1433   425  ...   
178  179    DET  2020    8257    8257   100.0  2075  1893  1341   463  ...   
179  180    STL  2020    8188    8188   100.0  2008  1752  1295   410  ...   

      BB   BB%    BA   OBP   SLG EV (MPH) LA (°) Dist (ft) Hard

In [35]:
# now that we have our dataframe full of batting stats across all 30 teams from the 2018-2023 seasons, we can start analyzing
# first we need to start changing columns into the datatypes we need for analysis
league_batter_stats.dtypes
# with this we see that all columns are objects right now

Rk.           object
Player        object
Year          object
Pitches       object
Total         object
Pitch %       object
PA            object
AB            object
BIP           object
Hits          object
1B            object
2B            object
3B            object
HR            object
SO            object
K%            object
BB            object
BB%           object
BA            object
OBP           object
SLG           object
EV (MPH)      object
LA (°)        object
Dist (ft)     object
Hard Hit%     object
Barrel/PA%    object
dtype: object

In [48]:
# columns that will be converted to integers: all except:
# Strings: Player, and Year
# Floats: Pitch %, K%, BB%, BA, OBP, SLG, EV, LA, HardHit%, Barrel/PA%
league_batter_stats['Player'] = league_batter_stats['Player'].astype('string')
league_batter_stats['Year'] = league_batter_stats['Year'].astype('string')
league_batter_stats[['Pitch %', 'K%', 'BB%', 'BA', 'OBP', 'SLG', 'EV (MPH)', 'LA (°)', 'Hard Hit%', 'Barrel/PA%']] = league_batter_stats[['Pitch %', 'K%', 'BB%', 'BA', 'OBP', 'SLG', 'EV (MPH)', 'LA (°)', 'Hard Hit%', 'Barrel/PA%']].astype('string')
league_batter_stats[['Pitch %', 'K%', 'BB%', 'BA', 'OBP', 'SLG', 'EV (MPH)', 'LA (°)', 'Hard Hit%', 'Barrel/PA%']] = league_batter_stats[['Pitch %', 'K%', 'BB%', 'BA', 'OBP', 'SLG', 'EV (MPH)', 'LA (°)', 'Hard Hit%', 'Barrel/PA%']].astype(float)
int_columns = ['Rk.', 'Pitches', 'Total', 'PA', 'AB', 'BIP', 'Hits', '1B', '2B', '3B', 'HR', 'SO', 'BB', 'Dist (ft)']
for x in int_columns:
    league_batter_stats[x] = league_batter_stats[x].astype('string')
    if (x == 'Pitches') or (x == 'Total'):
        league_batter_stats[x] = pd.to_numeric(league_batter_stats[x].str.replace(',',''), errors='coerce')
    else:
        league_batter_stats[x] = pd.to_numeric(league_batter_stats[x], errors='coerce')
print(league_batter_stats.dtypes)
print(league_batter_stats['Pitches'].head())

Rk.             int64
Player         string
Year           string
Pitches         int64
Total           int64
Pitch %       float64
PA              int64
AB              int64
BIP             int64
Hits            int64
1B              int64
2B              int64
3B              int64
HR              int64
SO              int64
K%            float64
BB              int64
BB%           float64
BA            float64
OBP           float64
SLG           float64
EV (MPH)      float64
LA (°)        float64
Dist (ft)       int64
Hard Hit%     float64
Barrel/PA%    float64
dtype: object
0    25643
1    25523
2    25427
3    25312
4    25306
Name: Pitches, dtype: int64
